# Project 1.2 - Multi-Stock EDA on Nifty 50 Companies

This project performs an exploratory data analysis (EDA) across all Nifty 50 companies using their historical stock data. The goal is to identify comparative stock behaviours, trading activity patterns, volatility, and trends across multiple comapnies over time. The dataset comprises daily data for each companies, merged into a unified format for multi stock insights.  

## Step 1: Dataset Consolidation

All 52 CSV files representing Nifty 50 companies have been successfully loaded and merged into a single DataFrame. Each record now includes a 'stock' column to retain company identity during analysis.

This unified format will enable comaparative,aggregate, and company-specific time series analysis with ease, helping us explore sectoral behaviour, stock-wise anomalies, and broader market trends.

**Explanation:** 
- csv_files: list[str], A list of file paths (strings) like 'folder/stock.csv'. Its a List comprehension.
- file: One file path at a time from the list
- df: DataFrame, Loaded data for eac stock using pd.read_csv(files)
- import os: Imports Python's built-in module to interact with the file system.Used for listing files in a folder.
- os.path.join('nifty_data', file):Creates a list of all the file paths by joing them by checking if the file ends with 'csv' by the **if** conditon
- pd.concat: combines all Stock DataFrames into one . ignore_index=True resets the row index after stacking .
- The stacking is like FIFO
  - file_1 rows
  - file_2 rows
  - file_3 rows
  - ...
  - file_52 rows
- df.dropna(): removes entire rows where at least one column has a missing value by default.<br>
  But it has options:
  - df.dropna(how='any'): removes rows if **any** column is NaN(default)
  - df.dropna(how='all'): removes rows only if **all** columns are NaN
  - df.dropna(subset=['Open', 'Close']): drops rows **only if these specific** columns are NaN.


In [1]:
import pandas as pd
import os

#Define path: its in the same directory as cwd


# List of all CSV files 
csv_files = [os.path.join('nifty_data', file) for file in os.listdir('nifty_data') if file.endswith('.csv')]

#Load each file into a list of DataFrames, adding a 'Stock' columnn and also ensure that no empty or Nan values are concatenated as it is 
# deprecated in future version of pandas
dataframes = []

for file in csv_files:
    df = pd.read_csv(file)

    #Clean empty rows and columns
    df.dropna(how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)

    #Add a column with the stock name
    stock_name = file.replace(".csv", "")
    df["Stock"] = stock_name
    if not df.empty:
        dataframes.append(df)
         
#Concatenate into one master DataFrame
nifty50_df = pd.concat(dataframes, ignore_index=True)

#Preview
nifty50_df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,Stock,Company Name,Industry,ISIN Code
0,2003-07-09,MARUTI,EQ,125.00,164.90,170.40,155.00,164.0,164.30,165.95,35164283.0,5.835528e+14,NaN,8537695.0,0.2428,nifty_data/MARUTI,NaN,NaN,NaN
1,2003-07-10,MARUTI,EQ,164.30,167.00,168.70,164.50,167.0,167.00,166.74,10464179.0,1.744820e+14,NaN,4363947.0,0.4170,nifty_data/MARUTI,NaN,NaN,NaN
2,2003-07-11,MARUTI,EQ,167.00,167.75,174.85,166.25,173.6,173.35,172.45,11740117.0,2.024622e+14,NaN,3014852.0,0.2568,nifty_data/MARUTI,NaN,NaN,NaN
3,2003-07-14,MARUTI,EQ,173.35,174.25,179.25,174.25,178.6,177.95,177.91,5982324.0,1.064313e+14,NaN,1949217.0,0.3258,nifty_data/MARUTI,NaN,NaN,NaN
4,2003-07-15,MARUTI,EQ,177.95,200.00,200.00,173.00,176.3,176.20,176.88,6173689.0,1.092001e+14,NaN,1307694.0,0.2118,nifty_data/MARUTI,NaN,NaN,NaN


In [2]:
# Correcting the name of %Deliverble to %Deliverable
nifty50_df.rename(columns={'%Deliverble' : '%Deliverable'}, inplace=True)
nifty50_df.head(1)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverable,Stock,Company Name,Industry,ISIN Code
0,2003-07-09,MARUTI,EQ,125.0,164.9,170.4,155.0,164.0,164.3,165.95,35164283.0,5.835528e+14,NaN,8537695.0,0.2428,nifty_data/MARUTI,NaN,NaN,NaN


## Step 2: Intial Exploration & Precprocessing
### What we'll Do:
1. View dataset shape and data types
2. Check for missing values
3. Explore unique stock symbols
4. Convert 'Date' column to to datetime
5. Sort data for time-series use


**Explanation :**
- shape: Gives us the number of rows and columns in the full dataset.
- dtypes: Shows the data type of each column (e.g.,object, float64, datetime)
- isnull().sum(): Helps us find how many missing values are there in each column.
- nunique() + unique():**[** Returns an array of all unique values in the column(including non-numeric values like strings or dates).**]**  **+**  **[** Returns the number (count) of unique values in that column.**]** (Checks how many different stocks are present and lists their names.)
- pd.to_datetime(): Converts the 'Date' column from string to pandas datetime, crucial for time-series.
- sort_values(): Ensures each stock's data is ordered by date . This is important for correct rolling metrics, trends etc.

In [3]:
# Step 2.1: Basic info and shape
print("Dataset shape:",nifty50_df.shape)
print("\nData Types:\n", nifty50_df.dtypes)
print("\nMissing Values:\n", nifty50_df.isnull().sum())

# Step 2.2: Check how many unique stocks
print("\nUnique Stocks :", nifty50_df['Symbol'].nunique())
print("\nList of stock Symbols:\n", nifty50_df['Symbol'].unique())

# Step 2.3: Convert 'DATE' to datetime
nifty50_df['Date'] = pd.to_datetime(nifty50_df['Date'])

# Step 2.4: - Sort by SYMBOL and DATE
nifty50_df = nifty50_df.sort_values(by=['Symbol' , 'Date']).reset_index(drop=True)


Dataset shape: (470434, 19)

Data Types:
 Date                   object
Symbol                 object
Series                 object
Prev Close            float64
Open                  float64
High                  float64
Low                   float64
Last                  float64
Close                 float64
VWAP                  float64
Volume                float64
Turnover              float64
Trades                float64
Deliverable Volume    float64
%Deliverable          float64
Stock                  object
Company Name           object
Industry               object
ISIN Code              object
dtype: object

Missing Values:
 Date                      50
Symbol                     0
Series                     0
Prev Close                50
Open                      50
High                      50
Low                       50
Last                      50
Close                     50
VWAP                      50
Volume                    50
Turnover                  50
Trades  

### Insights:
- The dataset contains 470434 rows and 19 columns.
- There are 66 unique stock symbols.
- The columns like 'Company Name', 'ISIN Code', 'Industry' are totally having missing values.

#### Step 2.5: Adding Metadata into main DataFrame 'nifty50_df' using a metadataframe 'df' 

#### Code Explanation:
We use the .map() function in pandas to fill missing values in specific columns (Company Name, Industry, ISIN Code) of the main data frame (nifty50_df) by mapping from a reference Data Frame (df) using a common key column: 'Symbol'.
- df.set_index('Symbol'): this sets the 'Symbol' column in the df as the index-so now each company's symbol becomes the row label.This is essential because .map() works by looking up values using an index or dictionary.
- .get: This is a built-in function in python used to fetch a value for a given key from a dictionary. When combined like this, series.get acts like a dictionary-style key lookup. 
- .map(): is extremely efficient for value-matching based on a single key column('Symbol'). For every value in the symbol, it looks up the corresponding value (e.g.,Company Name) from the dictionary-like Series creted earlier and returns it. The result is assigned back to the nifty50_df['Company Name'].
  

In [4]:
# Set the 'Symbol' column as the index
df = df.set_index('Symbol', inplace=True)

In [5]:
# Use '.map()' with .get() for each column to pull values from the df
nifty50_df['Company Name'] =nifty50_df['Symbol'].map(df['Company Name'].get)
nifty50_df['Industry'] = nifty50_df['Symbol'].map(df['Industry'].get)
nifty50_df['ISIN Code'] = nifty50_df['Symbol'].map(df['ISIN Code'].get)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
nifty50_df

## Step 3: Daily Returns and Volatility Computation

In [ ]:
# Grouping by stock so calculations happen stock-wise
nifty50_df['Daily_Return'] = nifty50_df.groupby('Symbol')['Close'].transform(lambda x: x.pct_change())

# Rolling volatility (20-day standard deviaton of returns)
nifty50_df['Volatility_20D'] = nifty50_df.groupby('Symbol')['Daily_Return'].transform(lambda x: x.rolling(window=20).std())

In [ ]:
nifty50_df.head(10)

#### Top 10 most volatile stocks:

In [ ]:
# Step 1: Get average volatility for each stock
avg_vol = nifty50_df.groupby('Symbol')['Volatility_20D'].mean().sort_values(ascending=False)

# Step 2: Select top 10 most volatile stocks
top_10 = avg_vol.head(10).index.tolist()

# Step 3: Plot their volatility 
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(14,7))
for sym in top_10:
    nifty50_df[nifty50_df['Symbol'] == sym].set_index('Date')['Volatility_20D'].plot(label=sym)

plt.legend()
plt.title("Top 10 Most Volatile Stocks (20D Rolling Volatility)")
plt.ylabel("Volatility")
plt.show()

#### Heatmap (all stocks volatility)

In [ ]:
df_grouped = nifty50_df.groupby(['Date','Symbol'], as_index=False).agg({'Daily_Return': 'mean'})

# 1.Create a pivot table: rows = Date, columns = Symbol, values = Returns
returns_pivot = df_grouped.pivot(index='Date', columns='Symbol', values='Daily_Return')

# 2. Drop dates ith missing data
returns_pivot = returns_pivot.dropna()

# 3. Compute the correlation matrix
correlation_matrix = returns_pivot.corr

# 4. Plot Heatmap
plt.figure(figsize=(15, 12))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False, linewidths=0.5)
plt.title("Correlation Heatmap of Daily Returns Across Nifty Stocks")
plt.tight_layout()
plt.show()

In [ ]:
df_grouped.shape

In [ ]:
nifty50_df.to_csv("Nifty50_cleaned.csv", index=False)